In [ ]:
import cv2
import h5py
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat
from skimage import color
from skimage import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# train_tar = "/content/drive/MyDrive/train.tar"
# test_tar = "/content/drive/MyDrive/test.tar"

In [ ]:
# !tar -xvf "/content/drive/MyDrive/train.tar" -C "/content"

tar: /content/drive/MyDrive/train.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
# !tar -xvf "/content/drive/MyDrive/test.tar" -C "/content"

In [ ]:
# train_img = []
# for img in glob.glob("/content/train/*.png"):
#   temp = cv2.imread(img)
#   train_img.append(temp)

In [ ]:
# test_img = []
# for img in glob.glob("/content/test/*.png"):
#   temp = cv2.imread(img)
#   test_img.append(temp)

In [ ]:
# len(train_img)

33402

In [ ]:
# len(test_img)

13068

In [ ]:
def load_data(mat_path):
  data = loadmat(mat_path)
  return data['X'], data['y']

In [ ]:
X_train, y_train = load_data('/content/drive/MyDrive/archive/train_32x32.mat')
X_test, y_test = load_data('/content/drive/MyDrive/archive/test_32x32.mat')

In [ ]:
print(X_train.shape)
print(y_test.shape)

(73257, 32, 32, 3)
(26032,)


In [ ]:
#convert into proper format (l,w,h,#channel)
X_train,y_train = X_train.transpose((3,0,1,2)), y_train[:,0] #permute
X_test,y_test = X_test.transpose((3,0,1,2)), y_test[:,0]

In [ ]:
print(X_train.shape)
print(y_test.shape)

(73257, 32, 32, 3)
(26032,)


In [ ]:
num_img = X_train.shape[0] + X_test.shape[0]
print(num_img)

99289


In [ ]:
np.unique(y_train)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [ ]:
#assigning label 10 to 0 for ease of read
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
#train and validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state = 2)

In [ ]:
def grayscale(images):
    return np.expand_dims(np.dot(images, [0.2990, 0.5870, 0.1140]), axis=3) # NTSC formula: 0.299 ∙ Red + 0.587 ∙ Green + 0.114 ∙ Blue
#try opencv
g_train = grayscale(X_train).astype(np.float32)
g_test = grayscale(X_test).astype(np.float32)
g_val = grayscale(X_val).astype(np.float32)

In [ ]:
# Normalizing
# Liang et al. 2015 removing the per-pixel mean value calculated over the entire set.

mean = np.mean(g_train, axis=0)

# Calculate the std on the training data
std = np.std(g_train, axis=0)

# Subtract it equally from all splits
ng_train = (g_train - mean) / std
ng_test = (g_test - mean)  / std
ng_val = (g_val - mean) / std

In [ ]:
#one hot encode
ohe = OneHotEncoder().fit(y_train.reshape(-1,1))

In [ ]:
y_train = ohe.transform(y_train.reshape(-1, 1)).toarray()
y_test = ohe.transform(y_test.reshape(-1, 1)).toarray()
y_val = ohe.transform(y_val.reshape(-1, 1)).toarray()

In [ ]:
fname = h5py.File('greyscale_svhn1.h5', 'w')
fname.create_dataset('X_train', data=ng_train)
fname.create_dataset('y_train', data=y_train)
fname.create_dataset('X_test', data=ng_test)
fname.create_dataset('y_test', data=y_test)
fname.create_dataset('X_val', data=ng_val)
fname.create_dataset('y_val', data=y_val)
fname.close()

In [ ]:
!pwd

/content


In [ ]:
!cp greyscale_svhn1.h5 /content/drive/MyDrive/archive